# Practical Application III: Comparing Classifiers

**Overview**: In this practical application, your goal is to compare the performance of the classifiers we encountered in this section, namely K Nearest Neighbor, Logistic Regression, Decision Trees, and Support Vector Machines.  We will utilize a dataset related to marketing bank products over the telephone.  



### Getting Started

Our dataset comes from the UCI Machine Learning repository [link](https://archive.ics.uci.edu/ml/datasets/bank+marketing).  The data is from a Portugese banking institution and is a collection of the results of multiple marketing campaigns.  We will make use of the article accompanying the dataset [here](CRISP-DM-BANK.pdf) for more information on the data and features.



### Problem 1: Understanding the Data

To gain a better understanding of the data, please read the information provided in the UCI link above, and examine the **Materials and Methods** section of the paper.  How many marketing campaigns does this data represent?

In [224]:
#17 Campaigns

### Problem 2: Read in the Data

Use pandas to read in the dataset `bank-additional-full.csv` and assign to a meaningful variable name.

In [225]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy 
import time
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

In [226]:
df = pd.read_csv('data/bank-additional-full.csv', sep = ';')

In [227]:
df.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


### Problem 3: Understanding the Features


Examine the data description below, and determine if any of the features are missing values or need to be coerced to a different data type.


```
Input variables:
# bank client data:
1 - age (numeric)
2 - job : type of job (categorical: 'admin.','blue-collar','entrepreneur','housemaid','management','retired','self-employed','services','student','technician','unemployed','unknown')
3 - marital : marital status (categorical: 'divorced','married','single','unknown'; note: 'divorced' means divorced or widowed)
4 - education (categorical: 'basic.4y','basic.6y','basic.9y','high.school','illiterate','professional.course','university.degree','unknown')
5 - default: has credit in default? (categorical: 'no','yes','unknown')
6 - housing: has housing loan? (categorical: 'no','yes','unknown')
7 - loan: has personal loan? (categorical: 'no','yes','unknown')
# related with the last contact of the current campaign:
8 - contact: contact communication type (categorical: 'cellular','telephone')
9 - month: last contact month of year (categorical: 'jan', 'feb', 'mar', ..., 'nov', 'dec')
10 - day_of_week: last contact day of the week (categorical: 'mon','tue','wed','thu','fri')
11 - duration: last contact duration, in seconds (numeric). Important note: this attribute highly affects the output target (e.g., if duration=0 then y='no'). Yet, the duration is not known before a call is performed. Also, after the end of the call y is obviously known. Thus, this input should only be included for benchmark purposes and should be discarded if the intention is to have a realistic predictive model.
# other attributes:
12 - campaign: number of contacts performed during this campaign and for this client (numeric, includes last contact)
13 - pdays: number of days that passed by after the client was last contacted from a previous campaign (numeric; 999 means client was not previously contacted)
14 - previous: number of contacts performed before this campaign and for this client (numeric)
15 - poutcome: outcome of the previous marketing campaign (categorical: 'failure','nonexistent','success')
# social and economic context attributes
16 - emp.var.rate: employment variation rate - quarterly indicator (numeric)
17 - cons.price.idx: consumer price index - monthly indicator (numeric)
18 - cons.conf.idx: consumer confidence index - monthly indicator (numeric)
19 - euribor3m: euribor 3 month rate - daily indicator (numeric)
20 - nr.employed: number of employees - quarterly indicator (numeric)

Output variable (desired target):
21 - y - has the client subscribed a term deposit? (binary: 'yes','no')
```



### Problem 4: Understanding the Task

After examining the description and data, your goal now is to clearly state the *Business Objective* of the task.  State the objective below.

In [228]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             41188 non-null  int64  
 1   job             41188 non-null  object 
 2   marital         41188 non-null  object 
 3   education       41188 non-null  object 
 4   default         41188 non-null  object 
 5   housing         41188 non-null  object 
 6   loan            41188 non-null  object 
 7   contact         41188 non-null  object 
 8   month           41188 non-null  object 
 9   day_of_week     41188 non-null  object 
 10  duration        41188 non-null  int64  
 11  campaign        41188 non-null  int64  
 12  pdays           41188 non-null  int64  
 13  previous        41188 non-null  int64  
 14  poutcome        41188 non-null  object 
 15  emp.var.rate    41188 non-null  float64
 16  cons.price.idx  41188 non-null  float64
 17  cons.conf.idx   41188 non-null 

In [229]:
labelencoder_X = LabelEncoder()
df_client['job']      = labelencoder_X.fit_transform(df_client['job']) 
df_client['marital']  = labelencoder_X.fit_transform(df_client['marital']) 
df_client['education']= labelencoder_X.fit_transform(df_client['education']) 
df_client['default']  = labelencoder_X.fit_transform(df_client['default']) 
df_client['housing']  = labelencoder_X.fit_transform(df_client['housing']) 
df_client['loan']     = labelencoder_X.fit_transform(df_client['loan']) 

### Problem 5: Engineering Features

Now that you understand your business objective, we will build a basic model to get started.  Before we can do this, we must work to encode the data.  Using just the bank information features (columns 1 - 7), prepare the features and target column for modeling with appropriate encoding and transformations.

In [230]:
df_client = df.iloc[: , 0:7]
df_client.head()

,age,job,marital,education,default,housing,loan
0,56,housemaid,married,basic.4y,no,no,no
1,57,services,married,high.school,unknown,no,no
2,37,services,married,high.school,no,yes,no
3,40,admin.,married,basic.6y,no,no,no
4,56,services,married,high.school,no,no,yes


In [231]:
labelencoder_X = LabelEncoder()
df_client['job']      = labelencoder_X.fit_transform(df_client['job']) 
df_client['marital']  = labelencoder_X.fit_transform(df_client['marital']) 
df_client['education']= labelencoder_X.fit_transform(df_client['education']) 
df_client['default']  = labelencoder_X.fit_transform(df_client['default']) 
df_client['housing']  = labelencoder_X.fit_transform(df_client['housing']) 
df_client['loan']     = labelencoder_X.fit_transform(df_client['loan']) 

In [232]:
def age(dataframe):
    dataframe.loc[dataframe['age'] <= 32, 'age'] = 1
    dataframe.loc[(dataframe['age'] > 32) & (dataframe['age'] <= 47), 'age'] = 2
    dataframe.loc[(dataframe['age'] > 47) & (dataframe['age'] <= 70), 'age'] = 3
    dataframe.loc[(dataframe['age'] > 70) & (dataframe['age'] <= 98), 'age'] = 4
           
    return dataframe

age(df_client);

df_client.head()

,age,job,marital,education,default,housing,loan
0,3,3,1,0,0,0,0
1,3,7,1,3,1,0,0
2,2,7,1,3,0,2,0
3,2,0,1,1,0,0,0
4,3,7,1,3,0,0,2


In [233]:
# Slicing DataFrame to treat separately, make things more easy
df_related = df.iloc[: , 7:11]

In [234]:
df_related['contact']     = labelencoder_X.fit_transform(df_related['contact']) 
df_related['month']       = labelencoder_X.fit_transform(df_related['month']) 
df_related['day_of_week'] = labelencoder_X.fit_transform(df_related['day_of_week'])

df_related.head()

,contact,month,day_of_week,duration
0,1,6,1,261
1,1,6,1,149
2,1,6,1,226
3,1,6,1,151
4,1,6,1,307


In [235]:
def duration(data):

    data.loc[data['duration'] <= 102, 'duration'] = 1
    data.loc[(data['duration'] > 102) & (data['duration'] <= 180)  , 'duration']    = 2
    data.loc[(data['duration'] > 180) & (data['duration'] <= 319)  , 'duration']   = 3
    data.loc[(data['duration'] > 319) & (data['duration'] <= 644.5), 'duration'] = 4
    data.loc[data['duration']  > 644.5, 'duration'] = 5

    return data
duration(df_related);

In [236]:
df_se = df.loc[: , ['emp.var.rate', 'cons.price.idx', 'cons.conf.idx', 'euribor3m', 'nr.employed']]
df_se.head()

,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed
0,1.1,93.994,-36.4,4.857,5191.0
1,1.1,93.994,-36.4,4.857,5191.0
2,1.1,93.994,-36.4,4.857,5191.0
3,1.1,93.994,-36.4,4.857,5191.0
4,1.1,93.994,-36.4,4.857,5191.0


In [237]:
df_other = df.loc[: , ['campaign', 'pdays','previous', 'poutcome']]
df_other.head()

,campaign,pdays,previous,poutcome
0,1,999,0,nonexistent
1,1,999,0,nonexistent
2,1,999,0,nonexistent
3,1,999,0,nonexistent
4,1,999,0,nonexistent


In [238]:
df_other['poutcome'].replace(['nonexistent', 'failure', 'success'], [1,2,3], inplace  = True)

In [239]:
df_final= pd.concat([df_client, df_related, df_se, df_other], axis = 1)
df_final = df_final[['age', 'job', 'marital', 'education', 'default', 'housing', 'loan',
                     'contact', 'month', 'day_of_week', 'duration', 'emp.var.rate', 'cons.price.idx', 
                     'cons.conf.idx', 'euribor3m', 'nr.employed', 'campaign', 'pdays', 'previous', 'poutcome']]
df_final.shape

(41188, 20)

### Problem 6: Train/Test Split

With your data prepared, split it into a train and test set.

In [240]:
y = df["y"].map({"no":0, "yes":1})
X_train, X_test, y_train, y_test = train_test_split(df_final, y, stratify=y, test_size=.3)

k_fold = KFold(n_splits=10, shuffle=True, random_state=0)

### Problem 7: A Baseline Model

Before we build our first model, we want to establish a baseline.  What is the baseline performance that our classifier should aim to beat?

In [241]:
sc_X = StandardScaler()
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.transform(X_test)

### Problem 8: A Simple Model

Use Logistic Regression to build a basic model on your data.  

In [242]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [243]:
lr = LogisticRegression() 
lr_t_start = time.time()
lr.fit(X_train,y_train)
lr_time = round(time.time()-knn_t_start, 5)
lr_pred = lr.predict(X_test)
lr_acc = round(accuracy_score(y_test, lr_pred),2)*100

lr_cv = round(cross_val_score(lr, X_train, y_train, cv=k_fold, n_jobs=1, scoring = 'accuracy').mean(),2)*100

Train Time: 11092.8658
Train Accuracy: 91.0
Test Accuracy: 91.0


### Problem 9: Score the Model

What is the accuracy of your model?

In [250]:
print("Train Time:", lr_time)
print("Train Accuracy:", lr_acc)
print("Test Accuracy:", lr_cv)

Train Time: 11092.8658
Train Accuracy: 91.0
Test Accuracy: 91.0


### Problem 10: Model Comparisons

Now, we aim to compare the performance of the Logistic Regression model to our KNN algorithm, Decision Tree, and SVM models.  Using the default settings for each of the models, fit and score each.  Also, be sure to compare the fit time of each of the models.  Present your findings in a `DataFrame` similar to that below:

| Model | Train Time | Train Accuracy | Test Accuracy |
| ----- | ---------- | -------------  | -----------   |
|     |    |.     |.     |

In [245]:
#KNN
knn = KNeighborsClassifier(n_neighbors=20)
knn_t_start = time.time()
knn.fit(X_train, y_train)
knn_time = round(time.time()-knn_t_start, 5)
knn_pred = knn.predict(X_test)
knn_acc = round(accuracy_score(y_test, knn_pred),2)*100

knn_cv = round(cross_val_score(knn, X_train, y_train, cv=k_fold, n_jobs=1, scoring = 'accuracy').mean(),2)*100

print("Train Time:", knn_time)
print("Train Accuracy:", knn_acc)
print("Test Accuracy:", knn_cv)

Train Time: 0.00405
Train Accuracy: 90.0
Test Accuracy: 90.0


In [246]:
# Decision Tree
dt = DecisionTreeClassifier(criterion='gini') 
dt_t_start = time.time()
dt.fit(X_train, y_train)
dt_time = round(time.time()-dt_t_start, 5)
dt_pred = dt.predict(X_test)
dt_acc = round(accuracy_score(y_test, dt_pred),2)*100

dt_cv = round(cross_val_score(dtree, X_train, y_train, cv=k_fold, n_jobs=1, scoring = 'accuracy').mean(),2)*100

print("Train Time:", dt_time)
print("Train Accuracy:", dt_acc)
print("Test Accuracy:", dt_cv)

Train Time: 0.12655
Train Accuracy: 89.0
Test Accuracy: 88.0


In [247]:
svc= SVC(kernel = 'linear')
svc_t_start = time.time()
svc.fit(X_train, y_train)
svc_time = round(time.time()-dt_t_start, 5)
svc_pred = svc.predict(X_test)
svc_acc = round(accuracy_score(y_test, svc_pred),2)*100

svc_cv = round(cross_val_score(svc, X_train, y_train, cv=k_fold, n_jobs=1, scoring = 'accuracy').mean(),2)*100

print("Train Time:", svc_time)
print("Train Accuracy:", svc_acc)
print("Test Accuracy:", svc_cv)

Train Time: 24.93332
Train Accuracy: 89.0
Test Accuracy: 90.0


### Problem 11: Improving the Model

Now that we have some basic models on the board, we want to try to improve these.  Below, we list a few things to explore in this pursuit.

- More feature engineering and exploration.  For example, should we keep the gender feature?  Why or why not?
- Hyperparameter tuning and grid search.  All of our models have additional hyperparameters to tune and explore.  For example the number of neighbors in KNN or the maximum depth of a Decision Tree.  
- Adjust your performance metric

In [248]:
res_dict = {'model': ['Logistic Regression', 'KNN', 'DT' 'SVC'],
           'train score': [lr_acc, knn_acc, dt_acc, svc_acc],
           'test score': [lr_cv, knn_cv, dt_cv, svc_cv],
           'average fit time': [lr_time, knn_time, dt_time, svc_time]}

In [249]:
print(res_dict)

{'model': ['Logistic Regression', 'KNN', 'DTSVC'], 'train score': [91.0, 90.0, 89.0, 89.0], 'test score': [91.0, 90.0, 88.0, 90.0], 'average fit time': [11092.8658, 0.00405, 0.12655, 24.93332]}


##### Questions